In [ ]:
# default_exp etl

# etl

> Utilerias para extraer y transformar datos sobre delincuencia en la CDMX.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# export
import os
import glob
import itertools
from pathlib import Path
import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import timedelta, date, datetime
import seaborn as sns
import requests
import h3
from shapely.geometry import Polygon

In [ ]:
# export
DATA_PATH = "datos/"
DOWNLOADS_PATH = "datos/descargas/"

## get_carpetas_from_api

In [ ]:
# export
def get_carpetas_from_api(limit=100):
    """Regresa un GeoDataFrame con los primeros `limit` registros de la base abierta."""
    url = f'https://datos.cdmx.gob.mx/api/3/action/datastore_search?resource_id=48fcb848-220c-4af0-839b-4fd8ac812c0f&limit={limit}'
    r = requests.get(url, allow_redirects=True)
    records = r.json()['result']['records'] 
    records = pd.DataFrame(records)
    records.replace('NA', np.nan, inplace=True)
    records.dropna(subset=['longitud', 'latitud'], how='any', inplace=True)
    records = gpd.GeoDataFrame(records, geometry=gpd.points_from_xy(records.longitud, records.latitud))
    records = records.set_crs(epsg=4326)
    records['fecha_hechos'] = pd.to_datetime(records.fecha_hechos)
    return records

In [ ]:
carpetas = get_carpetas_from_api()
assert type(carpetas) == gpd.GeoDataFrame

## get_victimas_from_api

In [ ]:
# export
def get_victimas_from_api(limit=100):
    """Regresa un GeoDataFrame con los primeros `limit` registros de la base abierta de víctimas."""
    url = f'https://datos.cdmx.gob.mx/api/3/action/datastore_search?resource_id=d543a7b1-f8cb-439f-8a5c-e56c5479eeb5&limit={limit}'
    r = requests.get(url, allow_redirects=True)
    records = r.json()['result']['records'] 
    records = pd.DataFrame(records)
    records.replace('NA', np.nan, inplace=True)
    records.dropna(subset=['longitud', 'latitud'], how='any', inplace=True)
    records = gpd.GeoDataFrame(records, geometry=gpd.points_from_xy(records.longitud, records.latitud))
    records = records.set_crs(epsg=4326)
    records['FechaHecho'] = pd.to_datetime(records.FechaHecho)
    return records

In [ ]:
victimas = get_victimas_from_api()
assert type(victimas) == gpd.GeoDataFrame

## get_historico_carpetas

In [ ]:
# export
def get_historico_carpetas():
    """Regresa un GeoDataFrame con todos los registros de carpetas de investigación."""
    archivo = os.path.join(DOWNLOADS_PATH, 'carpetas_fiscalia.csv')
    url = "https://archivo.datos.cdmx.gob.mx/fiscalia-general-de-justicia/carpetas-de-investigacion-fgj-de-la-ciudad-de-mexico/carpetas_completa_julio_2021.csv"
    r = requests.get(url, allow_redirects=True)
    open(archivo, 'wb').write(r.content)
    records = pd.read_csv(archivo)
    records.replace('NA', np.nan, inplace=True)
    records.dropna(subset=['longitud', 'latitud'], how='any', inplace=True)
    records = gpd.GeoDataFrame(records, geometry=gpd.points_from_xy(records.longitud, records.latitud))
    records = records.set_crs(epsg=4326)
    records['fecha_hechos'] = pd.to_datetime(records.fecha_hechos)
    return records

In [ ]:
carpetas_todas = get_historico_carpetas()
assert type(carpetas_todas) == gpd.GeoDataFrame

/tmp/ipykernel_22689/329436171.py:1: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  carpetas_todas = get_historico_carpetas()


## get_historico_victimas

In [ ]:
# export
def get_historico_victimas():
    """Regresa un GeoDataFrame con todos los registros de victimas en carpetas de investigación."""
    archivo = os.path.join(DOWNLOADS_PATH, 'victimas_carpetas_fiscalia.csv')
    url = "https://archivo.datos.cdmx.gob.mx/fiscalia-general-de-justicia/victimas-en-carpetas-de-investigacion-fgj/victimas_completa_enero_2022.csv"
    r = requests.get(url, allow_redirects=True)
    open(archivo, 'wb').write(r.content)
    records = pd.read_csv(archivo)
    records.replace('NA', np.nan, inplace=True)
    records.dropna(subset=['longitud', 'latitud'], how='any', inplace=True)
    records = gpd.GeoDataFrame(records, geometry=gpd.points_from_xy(records.longitud, records.latitud))
    records = records.set_crs(epsg=4326)
    records['FechaHecho'] = pd.to_datetime(records.FechaHecho)
    return records

In [ ]:
victimas_todas = get_historico_victimas()
assert type(victimas_todas) == gpd.GeoDataFrame

## get_carpetas_desde_archivo

In [ ]:
# export
def get_carpetas_desde_archivo(archivo):
    """Regresa un GeoDataFrame con los registros leídos de un archivo"""
    records = pd.read_csv(archivo)
    records.replace('NA', np.nan, inplace=True)
    records.dropna(subset=['longitud', 'latitud'], how='any', inplace=True)
    records = gpd.GeoDataFrame(records, geometry=gpd.points_from_xy(records.longitud, records.latitud))
    records = records.set_crs(epsg=4326)
    records['fecha_hechos'] = pd.to_datetime(records.fecha_hechos)
    return records

In [ ]:
carpetas_todas = get_carpetas_desde_archivo("datos/descargas/carpetas_fiscalia.csv")
assert type(carpetas_todas) == gpd.GeoDataFrame

/tmp/ipykernel_22689/796177901.py:1: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  carpetas_todas = get_carpetas_desde_archivo("datos/descargas/carpetas_fiscalia.csv")


## get_victimas_desde_archivo

In [ ]:
# export
def get_victimas_desde_archivo(archivo):
    """Regresa un GeoDataFrame con los registros leídos de un archivo"""
    records = pd.read_csv(archivo)
    records.replace('NA', np.nan, inplace=True)
    records.dropna(subset=['longitud', 'latitud'], how='any', inplace=True)
    records = gpd.GeoDataFrame(records, geometry=gpd.points_from_xy(records.longitud, records.latitud))
    records = records.set_crs(epsg=4326)
    records['FechaHecho'] = pd.to_datetime(records.FechaHecho)
    return records

In [ ]:
carpetas_todas = get_victimas_desde_archivo("datos/descargas/victimas_carpetas_fiscalia.csv")
assert type(carpetas_todas) == gpd.GeoDataFrame

## agrega_ids_espaciales

In [ ]:
# export
def agrega_ids_espaciales(carpetas):
    """Agrega ids de colonias y cuadrantes a la base de carpetas."""
    shapes = os.path.join(DATA_PATH, 'criminologia_capas.gpkg')
    colonias = gpd.read_file(shapes, layer='colonias').drop(columns='colonia_geom_6362')
    cuadrantes = gpd.read_file(shapes, layer='cuadrantes')
    carpetas = (gpd.tools.sjoin(carpetas, colonias[['colonia_cve', 'colonia_nombre', 'municipio_cvegeo', 'geometry']])
                .drop(columns=['index_right'])
               )
    carpetas = (gpd.tools.sjoin(carpetas, cuadrantes[['cuadrante_id', 'geometry']])
                .drop(columns=['index_right']))
    return carpetas

In [ ]:
carpetas = agrega_ids_espaciales(carpetas)
assert 'colonia_cve' in carpetas.columns
assert 'cuadrante_id' in carpetas.columns
assert 'municipio_cvegeo' in carpetas.columns
victimas = agrega_ids_espaciales(victimas)
assert 'colonia_cve' in victimas.columns
assert 'cuadrante_id' in victimas.columns
assert 'municipio_cvegeo' in victimas.columns

## agregar_categorias_de_usuario

In [ ]:
# export
def agregar_categorias_de_usuario(carpetas, archivo_categorias="datos/categorias_incidentes.csv"):
    """Agrega una columna con categorías definidas por el usuario.
    
      Las categorías tienen que venir en un csv con columnas incidente y categoria que
      relacionen las categorías del usuario con la columna delitos de la base de carpetas.
    """
    if 'categoria' in carpetas.columns:
        carpetas = carpetas.drop(columns='categoria')
    if 'incidente' in carpetas.columns:
        carpetas = carpetas.drop(columns='incidente')
    categorias = pd.read_csv(archivo_categorias)
    carpetas = (carpetas
                .merge(categorias, left_on='delito', right_on='incidente', how='left')
                .drop(columns='incidente'))
    return carpetas

In [ ]:
carpetas = agregar_categorias_de_usuario(carpetas)
assert 'categoria' in carpetas.columns

## exporta_datos_visualizador

In [ ]:
# export
def exporta_datos_visualizador(carpetas, archivo_resultado):
    """ Escribe en archivo_resultado un csv para consumirse en el visualizador."""
    carpetas['lat'] = carpetas.geometry.y
    carpetas['long'] = carpetas.geometry.x
    carpetas = carpetas[['fecha_hechos', 'delito', 'categoria', 'municipio_cvegeo', 'colonia_cve', 
                         'cuadrante_id', 'categoria', 'lat', 'long']]
    carpetas.to_csv(archivo_resultado)

In [ ]:
exporta_datos_visualizador(carpetas, "datos/salidas/carpetas.csv")

## serie_de_tiempo_categoria

In [ ]:
# export
def serie_de_tiempo_categoria(carpetas, fecha_inicio, categoria, freq='M'):
    """ Regresa una serie de tiempo con los agregados por `freq` de la `categoria`.
        
        parameters:
        carpetas: incidentes, deben traer la columna categoria
        fecha_inicio: pd.datetime fecha del inicio de la serie
        categoria: nombre de la categoría a agregar (`agregar_categorias_de_usuario`)
        freq: frecuencia de agregación (https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases)
    """
    carpetas = carpetas.loc[carpetas.fecha_hechos >= fecha_inicio]
    carpetas = carpetas.loc[carpetas.categoria == categoria]
    serie = (carpetas
             .set_index('fecha_hechos')[['categoria']]
             .resample(freq)
             .size()
             .reset_index()
             .rename({0:categoria}, axis=1)
            )
    return serie

In [ ]:
serie = serie_de_tiempo_categoria(carpetas_todas, pd.to_datetime('01/01/2016'), 'Robo a pasajero')
serie

,fecha_hechos,Robo a pasajero
0,2016-01-31,310
1,2016-02-29,265
2,2016-03-31,275
3,2016-04-30,319
4,2016-05-31,256
...,...,...
62,2021-03-31,493
63,2021-04-30,472
64,2021-05-31,500
65,2021-06-30,435


## punto_to_hexid

In [ ]:
# export
def punto_to_hexid(punto, resolution):
    """Regresa el hexid (h3) del punto."""
    return h3.geo_to_h3(punto.y, punto.x, resolution)

## agrega_en_hexagonos

In [ ]:
# export
def agrega_en_hexagonos(puntos, resolution):
    """Regresa un GeoDataFrame con las cuentas de puntos agregadas en hexágonos.
    
       params:
       puntos: GeoDataFrame: los puntos a agregar
       resolution: int: la resolución en uber.h3
    """
    puntos.loc[:,'hex_id'] = puntos.loc[:,'geometry'].apply(punto_to_hexid, args=[resolution])
    by_hex = puntos.groupby('hex_id').size().reset_index()
    # by_hex['geometry'] = by_hex['hex_id'].apply(lambda hex_id: Polygon(h3.h3_to_geo_boundary(hex_id)))
    by_hex['geometry'] = by_hex['hex_id'].apply(lambda hex_id: Polygon([x[::-1] for x in h3.h3_to_geo_boundary(hex_id)]))
    by_hex = gpd.GeoDataFrame(by_hex).rename({0:'incidentes'}, axis=1).set_crs(epsg=4326) 
    return by_hex
    